In [68]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import (
    col, to_date, to_timestamp, date_format, datediff, months_between, last_day, date_add, date_sub, year, month,
    dayofmonth, dayofyear, hour, minute, second, trim, ltrim, rtrim, rpad, lpad, concat_ws, lower, upper, initcap,
    reverse, regexp_replace
)
from pyspark.sql.types import StructType, StringType, ArrayType, StructField, LongType

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

# Funciones de fecha y hora

* **to_date**: Convierte una cadena en formato de fecha a un tipo de dato Date.
* **to_timestamp**: Convierte una cadena en formato de fecha a un tipo de dato Timestamp.
* **date_format**: Convierte una fecha a una cadena en el formato especificado.
* **datediff**: Devuelve el número de días entre dos fechas.
* **months_between**: Devuelve el número de meses entre dos fechas.
* **last_day**: Devuelve el último día del mes de la fecha especificada.
* **date_add**: Devuelve la fecha agregando el número de días especificado.
* **date_sub**: Devuelve la fecha restando el número de días especificado.
* **year**: Devuelve el año de la fecha especificada.
* **month**: Devuelve el mes de la fecha especificada.
* **dayofmonth**: Devuelve el día del mes de la fecha especificada.
* **dayofyear**: Devuelve el día del año de la fecha especificada.
* **hour**: Devuelve la hora de la fecha especificada.
* **minute**: Devuelve el minuto de la fecha especificada.
* **second**: Devuelve el segundo de la fecha especificada.

In [2]:
data = spark.read.parquet(
    r"../../data/s9_data/calculo.parquet"
)

data.printSchema()

data.show(truncate=False)

root
 |-- nombre: string (nullable = true)
 |-- fecha_ingreso: string (nullable = true)
 |-- fecha_salida: string (nullable = true)
 |-- baja_sistema: string (nullable = true)

+------+-------------+------------+-------------------+
|nombre|fecha_ingreso|fecha_salida|baja_sistema       |
+------+-------------+------------+-------------------+
|Jose  |2021-01-01   |2021-11-14  |2021-10-14 15:35:59|
|Mayara|2021-02-06   |2021-11-25  |2021-11-25 10:35:55|
+------+-------------+------------+-------------------+



In [3]:
# to_date y to_timestamp

data2 = (
    data
    .select(
        to_date(col("fecha_ingreso"), format="yyyy-MM-dd").alias("fecha_ingreso"),
        to_date(col("fecha_salida"), format="yyyy-MM-dd").alias("fecha_salida"),
        to_timestamp(col("baja_sistema"), format="yyyy-MM-dd HH:mm:ss").alias("baja_sistema"),
    )
)

data2.show()

+-------------+------------+-------------------+
|fecha_ingreso|fecha_salida|       baja_sistema|
+-------------+------------+-------------------+
|   2021-01-01|  2021-11-14|2021-10-14 15:35:59|
|   2021-02-06|  2021-11-25|2021-11-25 10:35:55|
+-------------+------------+-------------------+



In [4]:
# date_format

data3 = (
    data2
    .select(
        date_format("fecha_ingreso", format="dd-MM-yyyy").alias("fecha_ingreso"),
        date_format("fecha_salida", format="dd-MM-yyyy").alias("fecha_salida"),
        date_format(col("baja_sistema"), format="dd-MM-yyyy").alias("baja_sistema"),
    )
)

data3.show()

+-------------+------------+------------+
|fecha_ingreso|fecha_salida|baja_sistema|
+-------------+------------+------------+
|   01-01-2021|  14-11-2021|  14-10-2021|
|   06-02-2021|  25-11-2021|  25-11-2021|
+-------------+------------+------------+



In [5]:
# datediff, months_between, last_day

data4 = (
    data
    .select(
        data.nombre,
        datediff(data.fecha_salida, data.fecha_ingreso).alias("dias"),
        months_between("fecha_salida", "fecha_ingreso").alias("meses"),
        last_day("fecha_salida").alias("ultimo_dia"),
    )
)

data4.show()

+------+----+-----------+----------+
|nombre|dias|      meses|ultimo_dia|
+------+----+-----------+----------+
|  Jose| 317|10.41935484|2021-11-30|
|Mayara| 292| 9.61290323|2021-11-30|
+------+----+-----------+----------+



In [6]:
# date_add, date_sub

data.select(
    col("fecha_ingreso"),
    col("fecha_salida"),
    date_add("fecha_ingreso", 10).alias("fecha_ingreso_+10"),
    date_sub("fecha_salida", 10).alias("fecha_salida_+10"),
).show()

+-------------+------------+-----------------+----------------+
|fecha_ingreso|fecha_salida|fecha_ingreso_+10|fecha_salida_+10|
+-------------+------------+-----------------+----------------+
|   2021-01-01|  2021-11-14|       2021-01-11|      2021-11-04|
|   2021-02-06|  2021-11-25|       2021-02-16|      2021-11-15|
+-------------+------------+-----------------+----------------+



In [7]:
# year, month, dayofmonth, dayofyear, hour, minute, second

data.select(
    col("fecha_ingreso"),
    col("fecha_salida"),
    col("baja_sistema"),
    year("fecha_ingreso").alias("year_ingreso"),
    month("fecha_ingreso").alias("month_ingreso"),
    dayofmonth("fecha_ingreso").alias("dayofmonth_ingreso"),
    dayofyear("fecha_ingreso").alias("dayofyear_ingreso"),
    hour("baja_sistema").alias("hour_baja_sistema"),
    minute("baja_sistema").alias("minute_baja_sistema"),
    second("baja_sistema").alias("second_baja_sistema"),
).show()

+-------------+------------+-------------------+------------+-------------+------------------+-----------------+-----------------+-------------------+-------------------+
|fecha_ingreso|fecha_salida|       baja_sistema|year_ingreso|month_ingreso|dayofmonth_ingreso|dayofyear_ingreso|hour_baja_sistema|minute_baja_sistema|second_baja_sistema|
+-------------+------------+-------------------+------------+-------------+------------------+-----------------+-----------------+-------------------+-------------------+
|   2021-01-01|  2021-11-14|2021-10-14 15:35:59|        2021|            1|                 1|                1|               15|                 35|                 59|
|   2021-02-06|  2021-11-25|2021-11-25 10:35:55|        2021|            2|                 6|               37|               10|                 35|                 55|
+-------------+------------+-------------------+------------+-------------+------------------+-----------------+-----------------+---------------

# Funciones para trabajar con strings

* **trim**: Elimina los espacios en blanco del inicio y final de una cadena.
* **ltrim**: Elimina los espacios en blanco del inicio de una cadena.
* **rtrim**: Elimina los espacios en blanco del final de una cadena.
* **rpad**: Rellena una cadena con espacios en blanco o con el carácter especificado hasta que tenga la longitud especificada.
* **lpad**: Rellena una cadena con espacios en blanco o con el carácter especificado hasta que tenga la longitud especificada.
* **concat_ws**: Concatena las cadenas especificadas usando el separador especificado.
* **lower**: Convierte una cadena a minúsculas.
* **upper**: Convierte una cadena a mayúsculas.
* **initcap**: Convierte la primera letra de cada palabra de una cadena a mayúsculas.
* **reverse**: Invierte una cadena.
* **regexp_replace**: Reemplaza todas las subcadenas que coinciden con la expresión regular especificada por la cadena especificada.

In [8]:
df = spark.read.parquet(
    r"../../data/s9_data/data.parquet"
)

In [9]:
df.show(truncate=False)

+-------+
|nombre |
+-------+
| Spark |
+-------+



In [10]:
# trim, ltrim, rtrim

df.select(
    col("nombre"),
    trim(col("nombre")).alias("trim"),
    ltrim(col("nombre")).alias("ltrim"),
    rtrim(col("nombre")).alias("rtrim"),
).show()

+-------+-----+------+------+
| nombre| trim| ltrim| rtrim|
+-------+-----+------+------+
| Spark |Spark|Spark | Spark|
+-------+-----+------+------+



In [11]:
# rpda, lpad

df.select(
    col("nombre"),
    trim(rpad(col("nombre"), 10, " ")).alias("rpad"),
    trim(lpad(col("nombre"), 10, " ")).alias("lpad"),
    trim(rpad(col("nombre"), 10, "=")).alias("rpad_="),
    trim(lpad(col("nombre"), 10, "-")).alias("lpad_-"),
).show()

+-------+-----+-----+---------+---------+
| nombre| rpad| lpad|   rpad_=|   lpad_-|
+-------+-----+-----+---------+---------+
| Spark |Spark|Spark|Spark ===|--- Spark|
+-------+-----+-----+---------+---------+



In [12]:
df = spark.createDataFrame(
    data=[
        ("spark", "es", "genial")
    ],
    schema=["sujeto", "verbo", "adjetivo"])

df.show()

+------+-----+--------+
|sujeto|verbo|adjetivo|
+------+-----+--------+
| spark|   es|  genial|
+------+-----+--------+



In [13]:
# concat_ws, lower, upper, initcap, reverse

df.select(
    concat_ws(" ", col("sujeto"), col("verbo"), col("adjetivo")).alias("concat_ws"),
    lower(col("sujeto")).alias("lower"),
    upper(col("sujeto")).alias("upper"),
    initcap(col("sujeto")).alias("initcap"),
    reverse(col("sujeto")).alias("reverse"),
).show()

+---------------+-----+-----+-------+-------+
|      concat_ws|lower|upper|initcap|reverse|
+---------------+-----+-----+-------+-------+
|spark es genial|spark|SPARK|  Spark|  kraps|
+---------------+-----+-----+-------+-------+



In [14]:
# regexp_replace

df = spark.createDataFrame(
    data=[
        (" voy a casa a por mis llaves",)
    ],
    schema=["frase"])

df.show(truncate=False)

+----------------------------+
|frase                       |
+----------------------------+
| voy a casa a por mis llaves|
+----------------------------+



In [15]:
# regexp_replace

df.select(
    col("frase"),
    regexp_replace(col("frase"), "voy|por", "ir").alias("regexp_replace"),
).show(truncate=False)

+----------------------------+--------------------------+
|frase                       |regexp_replace            |
+----------------------------+--------------------------+
| voy a casa a por mis llaves| ir a casa a ir mis llaves|
+----------------------------+--------------------------+



# Funciones para trabajar con colecciones

* **size**: Devuelve el tamaño de la colección.

* **sort_array**: Ordena los elementos de la colección en orden ascendente o descendente según el parámetro especificado. Si no se especifica el parámetro, se ordena en orden ascendente. Los elementos han de ser del mismo tipo.

* **array_contains**: Devuelve true si el array contiene el valor especificado.

* **explode**: Crea una fila por cada elemento de la colección.

* **to_json**: Se utiliza para convertir una columna o estructura de datos en formato JSON. Toma una columna o una expresión de estructura y devuelve una cadena de caracteres en formato JSON que representa los datos. Esto es útil cuando quieres convertir datos estructurados en un formato que sea compatible con JSON.

* **from_json**: Se utiliza para convertir una columna de tipo JSON en una estructura de datos anidada de PySpark. Toma una columna JSON y un esquema (schema) de PySpark como argumentos y devuelve una nueva columna que contiene la estructura de datos anidada. Esta función es útil cuando quieres extraer datos de una cadena JSON y utilizarlos en consultas o transformaciones posteriores.

* **get_json_object**: Se utiliza para extraer un valor específico de una cadena JSON. Toma una columna JSON y una ruta (path) en formato de cadena como argumentos, y devuelve el valor correspondiente a esa ruta en la cadena JSON. Puedes utilizar esta función para acceder a valores específicos dentro de una estructura de datos JSON sin tener que convertir toda la cadena JSON en una estructura anidada.

In [2]:
from pyspark.sql.functions import size, sort_array, array_contains, explode

In [3]:
df = spark.read.parquet(
    r"../../data/s9_data/parquet"
)

df.show(truncate=False)

+-----+--------------------------------------------+
|dia  |tareas                                      |
+-----+--------------------------------------------+
|lunes|[hacer la tarea, buscar agua, lavar el auto]|
+-----+--------------------------------------------+



In [4]:
# size

df.select(
    col("dia"),
    size(col("tareas")),
).show(truncate=False)

+-----+------------+
|dia  |size(tareas)|
+-----+------------+
|lunes|3           |
+-----+------------+



In [5]:
# sort_array

df.select(
    col("dia"),
    sort_array(col("tareas")).alias("tareas_ordenadas"),
).show(truncate=False)

+-----+--------------------------------------------+
|dia  |tareas_ordenadas                            |
+-----+--------------------------------------------+
|lunes|[buscar agua, hacer la tarea, lavar el auto]|
+-----+--------------------------------------------+



In [6]:
# array_contains

df.select(
    col("dia"),
    array_contains(col("tareas"), "limpiar").alias("limpiar"),
).show(truncate=False)

df.select(
    col("dia"),
    col("tareas"),
    array_contains(col("tareas"), "lavar el auto").alias("lavar el auto"),
).show(truncate=False)

+-----+-------+
|dia  |limpiar|
+-----+-------+
|lunes|false  |
+-----+-------+

+-----+--------------------------------------------+-------------+
|dia  |tareas                                      |lavar el auto|
+-----+--------------------------------------------+-------------+
|lunes|[hacer la tarea, buscar agua, lavar el auto]|true         |
+-----+--------------------------------------------+-------------+



In [8]:
# explode

df.select(
    col("dia"),
    explode(col("tareas")).alias("tarea"),
).show(truncate=False)

+-----+--------------+
|dia  |tarea         |
+-----+--------------+
|lunes|hacer la tarea|
|lunes|buscar agua   |
|lunes|lavar el auto |
+-----+--------------+



In [10]:
# JSON

from pyspark.sql.functions import from_json, get_json_object, to_json

df = spark.read.parquet(
    r"../../data/s9_data/JSON"
)

df.show(truncate=False)

df.printSchema()

+---------------------------------------------------------------------------+
|tareas_str                                                                 |
+---------------------------------------------------------------------------+
|{"dia": "lunes","tareas": ["hacer la tarea","buscar agua","lavar el auto"]}|
+---------------------------------------------------------------------------+

root
 |-- tareas_str: string (nullable = true)



In [21]:
# from_json

schema = StructType([
    StructField("dia", StringType(), nullable=True),
    StructField("tareas", ArrayType(StringType()), nullable=True),
])

json_df = df.select(
    from_json(col("tareas_str"), schema=schema).alias("tareas_json"),
)

json_df.printSchema()

json_df.show(truncate=False)

root
 |-- tareas_json: struct (nullable = true)
 |    |-- dia: string (nullable = true)
 |    |-- tareas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)

+-----------------------------------------------------+
|tareas_json                                          |
+-----------------------------------------------------+
|{lunes, [hacer la tarea, buscar agua, lavar el auto]}|
+-----------------------------------------------------+



In [28]:
json_df.select(
    col("tareas_json").getItem("dia").alias("dia"),
    col("tareas_json").getItem("tareas").alias("tareas"),
    col("tareas_json").getItem("tareas").getItem(0).alias("tarea_0"),
    col("tareas_json").getItem("tareas").getItem(1).alias("tarea_1"),
    col("tareas_json").getItem("tareas").getItem(2).alias("tarea_2"),
).show(truncate=False)

+-----+--------------------------------------------+--------------+-----------+-------------+
|dia  |tareas                                      |tarea_0       |tarea_1    |tarea_2      |
+-----+--------------------------------------------+--------------+-----------+-------------+
|lunes|[hacer la tarea, buscar agua, lavar el auto]|hacer la tarea|buscar agua|lavar el auto|
+-----+--------------------------------------------+--------------+-----------+-------------+



In [37]:
# to_json

json_df.show(truncate=False)

to_json_df = json_df.select(
    to_json(col("tareas_json")).alias("tareas_str"),
)

to_json_df.show(truncate=False)

+-----------------------------------------------------+
|tareas_json                                          |
+-----------------------------------------------------+
|{lunes, [hacer la tarea, buscar agua, lavar el auto]}|
+-----------------------------------------------------+

+-------------------------------------------------------------------------+
|tareas_str                                                               |
+-------------------------------------------------------------------------+
|{"dia":"lunes","tareas":["hacer la tarea","buscar agua","lavar el auto"]}|
+-------------------------------------------------------------------------+



In [38]:
# get_json_object

to_json_df.select(
    get_json_object(col("tareas_str"), "$.dia").alias("dia"),
    get_json_object(col("tareas_str"), "$.tareas").alias("tareas"),
).show(truncate=False)

+-----+------------------------------------------------+
|dia  |tareas                                          |
+-----+------------------------------------------------+
|lunes|["hacer la tarea","buscar agua","lavar el auto"]|
+-----+------------------------------------------------+



# Funciones when, coalesce y lit

* **when**: Se utiliza para aplicar condiciones lógicas a los datos. Toma una expresión booleana y un valor o una columna como argumentos. Si la expresión booleana se evalúa como verdadera, devuelve el valor o la columna especificada. Puedes usar when en combinación con otras funciones para aplicar transformaciones condicionales en tus datos.

* **coalesce**: Se utiliza para obtener el primer valor no nulo de una lista de columnas. Toma una lista de columnas y devuelve una nueva columna que contiene el primer valor no nulo de esa lista. Es útil cuando quieres obtener un valor de respaldo en caso de que una columna tenga valores nulos.

* **lit**: Se utiliza para crear una columna constante con un valor especificado. Toma un valor como argumento y devuelve una nueva columna que contiene ese valor constante. Puedes usar lit para agregar columnas con valores constantes a tus datos.

In [39]:
# when, coalesce, lit

from pyspark.sql.functions import when, coalesce, lit

df = spark.read.parquet(
    r"../../data/s9_data/l83"
)

df.printSchema()

df.show(truncate=False)

root
 |-- nombre: string (nullable = true)
 |-- pago: long (nullable = true)

+------+----+
|nombre|pago|
+------+----+
|Jose  |1   |
|Julia |2   |
|Katia |1   |
|null  |3   |
|Raul  |3   |
+------+----+



In [53]:
# when pago > 1 alto, when pago > 1 and pago < 2 medio, when pago < 1 bajo

df.select(
    col("nombre"),
    col("pago"),
    when(col("pago") >= 3, "alto").when((col("pago") > 1) & (col("pago") < 3), "medio").otherwise("bajo").alias(
        "nivel_pago"),
).show(truncate=False)

+------+----+----------+
|nombre|pago|nivel_pago|
+------+----+----------+
|Jose  |1   |bajo      |
|Julia |2   |medio     |
|Katia |1   |bajo      |
|null  |3   |alto      |
|Raul  |3   |alto      |
+------+----+----------+



In [55]:
# coalesce y lit

df.select(
    col("nombre"),
    col("pago"),
    coalesce(col("nombre"), lit("sin nombre")).alias("nombre_no_nulo"),
).show(truncate=False)

+------+----+--------------+
|nombre|pago|nombre_no_nulo|
+------+----+--------------+
|Jose  |1   |Jose          |
|Julia |2   |Julia         |
|Katia |1   |Katia         |
|null  |3   |sin nombre    |
|Raul  |3   |Raul          |
+------+----+--------------+



# Funciones definidas por el usuario (UDF)

Las User-Defined Functions (UDF) son funciones personalizadas definidas por el usuario que se pueden utilizar para realizar transformaciones complejas en los datos. Estas funciones permiten extender las funcionalidades de PySpark más allá de las funciones integradas y proporcionan flexibilidad para aplicar lógica personalizada a tus datos.

Aquí hay algunos puntos clave sobre las UDF en PySpark:

* **Definición**: Una UDF se define utilizando una función de Python regular. Puedes escribir la lógica personalizada dentro de esta función para procesar los datos según tus necesidades.

* **Registro de la UDF**: Después de definir una UDF, debes registrarla con PySpark para que pueda reconocerla y aplicarla a los datos. Para registrar una UDF, utilizas el método udf del módulo pyspark.sql.functions. Este método toma como argumento la función definida por el usuario y devuelve una UDF registrada.

* **Tipos de UDF**: Las UDF pueden ser clasificadas en dos categorías principales:

    * **UDF escalar**: Toma una o más columnas como entrada y devuelve un solo valor como resultado. Puedes aplicar una UDF escalar a una columna o a un conjunto de columnas para generar una nueva columna con los resultados de la función personalizada.

    * **UDF de columna**: Toma una o más columnas como entrada y devuelve una nueva columna como resultado. La UDF de columna se aplica a nivel de fila y puedes utilizarla para aplicar transformaciones más complejas que involucren múltiples columnas.

* **Aplicación de UDF**: Una vez registrada, puedes aplicar la UDF a tus datos utilizando las funciones de PySpark, como select, withColumn, groupBy, etc. Puedes llamar a la UDF dentro de estas funciones y pasar las columnas requeridas como argumentos.

Es importante tener en cuenta que el uso excesivo de UDF puede afectar el rendimiento, ya que las UDF implican la ejecución de código Python a nivel de fila en lugar de operaciones vectorizadas optimizadas. Por lo tanto, se recomienda utilizar las funciones integradas de PySpark siempre que sea posible y utilizar UDF solo cuando sea necesario para casos de lógica personalizada compleja.

In [57]:
def cubed(n: int) -> int:
    return n * n * n


spark.udf.register("cubed", cubed, LongType())

<function __main__.cubed(n: int) -> int>

In [58]:
spark.range(1, 100).createOrReplaceTempView("test")

In [60]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM test").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
|  9|     729|
| 10|    1000|
| 11|    1331|
| 12|    1728|
| 13|    2197|
| 14|    2744|
| 15|    3375|
| 16|    4096|
| 17|    4913|
| 18|    5832|
| 19|    6859|
| 20|    8000|
+---+--------+
only showing top 20 rows



In [72]:
from pyspark.sql.functions import udf

# def welcome(name: str) -> str:
#     return f"Welcome to PySpark {name}!"

welcome_udf = udf(lambda name: f"Welcome {name}!", StringType())

In [71]:
df_names = spark.createDataFrame(
    data=[("John",), ("James",), ("Anna",)],
    schema=["name"]
)

df_names.select(
    col("name"),
    welcome_udf(col("name")).alias("welcome")
).show()

+-----+--------------+
| name|       welcome|
+-----+--------------+
| John| Welcome John!|
|James|Welcome James!|
| Anna| Welcome Anna!|
+-----+--------------+



In [75]:
@udf(returnType=StringType())
def upper(text: str) -> str:
    return text.upper()

df_names.select(
    col("name"),
    upper(col("name")).alias("welcome")
).show(truncate=False)

+-----+-------+
|name |welcome|
+-----+-------+
|John |JOHN   |
|James|JAMES  |
|Anna |ANNA   |
+-----+-------+



## UDF vectorizadas o Pandas UDF

Las UDF vectorizadas o Pandas UDF (User-Defined Functions) son una extensión de las UDF en PySpark que permiten procesar datos de forma eficiente al aplicar operaciones en lotes utilizando la biblioteca Pandas.

En PySpark, las UDF vectorizadas aprovechan la capacidad de Pandas para procesar datos en lotes, lo que implica una ejecución más rápida en comparación con las UDF regulares que operan a nivel de fila. En lugar de procesar los datos fila por fila, las UDF vectorizadas procesan los datos en grupos (batches) y aprovechan las operaciones vectorizadas optimizadas de Pandas.

Aquí hay algunos puntos clave sobre las UDF vectorizadas o Pandas UDF:

* **Funcionamiento**: Las UDF vectorizadas toman como entrada un DataFrame de PySpark y aplican operaciones en lotes a los datos utilizando Pandas. El DataFrame de PySpark se divide en lotes más pequeños y cada lote se convierte en un DataFrame de Pandas para aplicar la lógica personalizada utilizando las capacidades vectorizadas de Pandas. Luego, los resultados se devuelven como un nuevo DataFrame de PySpark.

* **Eficiencia**: Al procesar datos en lotes utilizando Pandas, las UDF vectorizadas reducen la sobrecarga de comunicación entre Python y el entorno de ejecución de PySpark, lo que puede conducir a una mejora significativa en el rendimiento y la eficiencia en comparación con las UDF regulares. Además, Pandas ofrece una amplia gama de funciones y operaciones optimizadas para el procesamiento de datos, lo que puede acelerar aún más las transformaciones.

* **Tipos de UDF vectorizadas**: Al igual que las UDF regulares, las UDF vectorizadas también pueden ser UDF escalares o UDF de columna. Puedes aplicar una UDF vectorizada escalar a una o más columnas y generar una nueva columna con los resultados vectorizados. Por otro lado, una UDF vectorizada de columna toma un conjunto de columnas como entrada y devuelve un nuevo DataFrame con las columnas resultantes.

* **Restricciones**: Las UDF vectorizadas tienen algunas restricciones en cuanto a las operaciones compatibles y los tipos de datos. Al utilizar Pandas, las operaciones deben ser compatibles con el modelo de datos de Pandas y no se admiten todas las funciones y operaciones de PySpark. Además, las UDF vectorizadas no son adecuadas para todas las situaciones y pueden requerir más memoria, especialmente cuando se trabajan con grandes volúmenes de datos.

In [78]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

def pandas_cubed_udf(v: pd.Series) -> pd.Series:
    return v * v * v

pandas_udf(pandas_cubed_udf, returnType=LongType())

<function __main__.pandas_cubed_udf(v: pandas.core.series.Series) -> pandas.core.series.Series>

In [84]:
pd_series = pd.Series([1, 2, 3, 4, 5])

pd_series = pandas_cubed_udf(pd_series)

In [86]:
df.show(truncate=False)

df.select(
    pandas_cubed_udf(col("num")).alias("num_cubed")
).show(truncate=False)

+---+------+
|num|letter|
+---+------+
|1  |a     |
|2  |b     |
|3  |c     |
+---+------+

+---------+
|num_cubed|
+---------+
|1        |
|8        |
|27       |
+---------+



# Funciones de ventana (Window Functions) en PySpark

Las funciones de ventana (window functions) en PySpark son operaciones analíticas avanzadas que se aplican a un conjunto de filas en una ventana definida. Estas funciones permiten realizar cálculos y transformaciones basados en grupos de filas en lugar de operar en filas individuales, lo que facilita el análisis y la agregación de datos en ventanas específicas.

Aquí hay algunos puntos clave sobre las funciones de ventana en PySpark:

* **Definición de ventana**: Antes de aplicar una función de ventana, debes definir la ventana utilizando la cláusula window en combinación con las funciones de agregación o análisis. Una ventana se define mediante una especificación de partición (partition), ordenación (ordering) y un marco (frame). La especificación de partición divide los datos en grupos lógicos, la ordenación establece el orden de las filas dentro de cada grupo y el marco define las filas relativas utilizadas en los cálculos.

* **Funciones de ventana comunes**: PySpark proporciona una variedad de funciones de ventana integradas para realizar cálculos y transformaciones. Algunas funciones comunes incluyen rank, row_number, dense_rank, lag, lead, sum, avg, min, max, first_value y last_value. Estas funciones se aplican a las filas dentro de la ventana y generan resultados basados en la lógica definida.

* **Uso de funciones de ventana**: Las funciones de ventana se utilizan en combinación con la función over, que especifica la ventana a la que se aplicará la función. La función over se aplica a una columna y toma como argumento la definición de la ventana que se ha creado previamente. Puedes utilizar la función over junto con una función de ventana para generar una nueva columna con los resultados calculados.

* **Tipos de ventanas**: PySpark admite varios tipos de ventanas, como ventanas sin partición (unbounded), ventanas particionadas (partitioned), ventanas deslizantes (sliding), ventanas de rango (range) y ventanas de fila (row). Cada tipo de ventana tiene su propia lógica y sintaxis para definir la ventana y realizar los cálculos correspondientes.

## Funciones

* **row_number**: Asigna un número de fila a cada fila en una partición.
* **rank**: Asigna un rango a cada fila en una partición, pero si hay empates, los números de fila no son consecutivos.
* **dense_rank**: Asigna un rango a cada fila en una partición, pero si hay empates, los números de fila son consecutivos.

In [87]:
df = spark.read.parquet(
    r"../../data/s10/funciones_ventana.parquet"
)

df.printSchema()

df.show(truncate=False)

root
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- departamento: string (nullable = true)
 |-- evaluacion: integer (nullable = true)

+-------+----+------------+----------+
|nombre |edad|departamento|evaluacion|
+-------+----+------------+----------+
|Lazaro |45  |letras      |98        |
|Raul   |24  |matemática  |76        |
|Maria  |34  |matemática  |27        |
|Jose   |30  |química     |78        |
|Susana |51  |química     |98        |
|Juan   |44  |letras      |89        |
|Julia  |55  |letras      |92        |
|Kadir  |38  |arquitectura|39        |
|Lilian |23  |arquitectura|94        |
|Rosa   |26  |letras      |91        |
|Aian   |50  |matemática  |73        |
|Yaneisy|29  |letras      |89        |
|Enrique|40  |química     |92        |
|Jon    |25  |arquitectura|78        |
|Luisa  |39  |arquitectura|94        |
+-------+----+------------+----------+



In [92]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, desc

window_spec = Window.partitionBy("departamento").orderBy(desc(df.evaluacion))

In [95]:
# row_number

df.withColumn(
    "row_number",
    row_number().over(window_spec)
).filter(col("row_number") <= 2).show(truncate=False)

+-------+----+------------+----------+----------+
|nombre |edad|departamento|evaluacion|row_number|
+-------+----+------------+----------+----------+
|Lilian |23  |arquitectura|94        |1         |
|Luisa  |39  |arquitectura|94        |2         |
|Lazaro |45  |letras      |98        |1         |
|Julia  |55  |letras      |92        |2         |
|Raul   |24  |matemática  |76        |1         |
|Aian   |50  |matemática  |73        |2         |
|Susana |51  |química     |98        |1         |
|Enrique|40  |química     |92        |2         |
+-------+----+------------+----------+----------+



In [98]:
# rank

df.withColumn(
    "rank",
    rank().over(window_spec)
).filter(col("rank") <= 2).show(truncate=False)

+-------+----+------------+----------+----+
|nombre |edad|departamento|evaluacion|rank|
+-------+----+------------+----------+----+
|Lilian |23  |arquitectura|94        |1   |
|Luisa  |39  |arquitectura|94        |1   |
|Lazaro |45  |letras      |98        |1   |
|Julia  |55  |letras      |92        |2   |
|Raul   |24  |matemática  |76        |1   |
|Aian   |50  |matemática  |73        |2   |
|Susana |51  |química     |98        |1   |
|Enrique|40  |química     |92        |2   |
+-------+----+------------+----------+----+



In [99]:
# dense_rank

df.withColumn(
    "dense_rank",
    dense_rank().over(window_spec)
).filter(col("dense_rank") <= 2).show(truncate=False)

+-------+----+------------+----------+----------+
|nombre |edad|departamento|evaluacion|dense_rank|
+-------+----+------------+----------+----------+
|Lilian |23  |arquitectura|94        |1         |
|Luisa  |39  |arquitectura|94        |1         |
|Jon    |25  |arquitectura|78        |2         |
|Lazaro |45  |letras      |98        |1         |
|Julia  |55  |letras      |92        |2         |
|Raul   |24  |matemática  |76        |1         |
|Aian   |50  |matemática  |73        |2         |
|Susana |51  |química     |98        |1         |
|Enrique|40  |química     |92        |2         |
+-------+----+------------+----------+----------+



In [105]:
from pyspark.sql.functions import min, max, avg

window_spec_agg = Window.partitionBy("departamento")

(
    df
    .withColumn(
        "min_evaluacion",
        min(df.evaluacion).over(window_spec_agg),
    )
    .withColumn(
        "max_evaluacion",
        max(df.evaluacion).over(window_spec_agg),
    )
    .withColumn(
        "avg_evaluacion",
        avg(df.evaluacion).over(window_spec_agg),
    )
    .withColumn(
        "row_number",
        row_number().over(window_spec),
    )
    .show(truncate=False)
)

+-------+----+------------+----------+--------------+--------------+------------------+----------+
|nombre |edad|departamento|evaluacion|min_evaluacion|max_evaluacion|avg_evaluacion    |row_number|
+-------+----+------------+----------+--------------+--------------+------------------+----------+
|Lilian |23  |arquitectura|94        |39            |94            |76.25             |1         |
|Luisa  |39  |arquitectura|94        |39            |94            |76.25             |2         |
|Jon    |25  |arquitectura|78        |39            |94            |76.25             |3         |
|Kadir  |38  |arquitectura|39        |39            |94            |76.25             |4         |
|Lazaro |45  |letras      |98        |89            |98            |91.8              |1         |
|Julia  |55  |letras      |92        |89            |98            |91.8              |2         |
|Rosa   |26  |letras      |91        |89            |98            |91.8              |3         |
|Juan   |4

# Catalyst optimizer

Catalyst Optimizer es un componente fundamental en Apache Spark y, por lo tanto, también en PySpark. Es un sistema de optimización de consultas basado en reglas que se encarga de analizar, transformar y optimizar el plan de ejecución de las consultas en Spark. Su objetivo es generar un plan de ejecución eficiente que minimice el tiempo de procesamiento y utilice de manera óptima los recursos disponibles.

A continuación, se presentan algunas características clave del Catalyst Optimizer en PySpark:

* **Análisis y optimización lógica**: El Catalyst Optimizer realiza un análisis exhaustivo de la estructura lógica de las consultas y aplica diversas optimizaciones basadas en reglas. Examina la estructura de las expresiones, las relaciones entre tablas y las operaciones a realizar, buscando formas más eficientes de ejecutar la consulta.

* **Árbol de expresiones**: El Catalyst Optimizer utiliza un árbol de expresiones para representar las operaciones lógicas y físicas de una consulta. Este árbol permite una manipulación flexible y modular de las expresiones y las operaciones, lo que facilita la aplicación de reglas de optimización y la transformación del plan de ejecución.

* **Reglas de optimización**: Catalyst utiliza un conjunto de reglas de optimización predefinidas para reescribir y mejorar el plan de ejecución de la consulta. Estas reglas incluyen fusionar operaciones redundantes, eliminar operaciones innecesarias, cambiar el orden de ejecución de las operaciones y aplicar técnicas como la poda de columnas (column pruning) y la poda de filtros (filter pushdown) para reducir el tamaño de los datos a procesar.

* **Optimización física**: Además de las optimizaciones lógicas, el Catalyst Optimizer también realiza optimizaciones físicas. Determina la distribución de datos más eficiente para las operaciones y selecciona los algoritmos de ejecución más adecuados, como la selección de un join hash o un join de bucle anidado en función de las características de los datos y los recursos disponibles.

* **Extensibilidad**: El Catalyst Optimizer en PySpark es altamente extensible. Permite a los desarrolladores y usuarios agregar nuevas reglas de optimización personalizadas o modificar las reglas existentes para adaptarse a casos de uso específicos. Esto proporciona flexibilidad para ajustar y optimizar el plan de ejecución según los requisitos y las características de los datos.

In [107]:
df = spark.read.parquet(
    r"../../data/s10/vuelos.parquet"
)

df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [108]:
new_df = (
    df
    .filter(
        col("MONTH").isin(6, 7, 8),
    )
    .withColumn(
        "DIS_AIRTIME",
        col("DISTANCE") / col("AIR_TIME"),
    )
    .select(
        col("AIRLINE"),
        col("DIS_AIRTIME"),
    )
    .where(
        col("AIRLINE").isin("AA", "AS", "DL", "UA"),
    )
)

new_df.show(truncate=False)

+-------+-----------------+
|AIRLINE|DIS_AIRTIME      |
+-------+-----------------+
|AA     |7.649769585253456|
|DL     |8.14176245210728 |
|DL     |8.361344537815127|
|DL     |8.358851674641148|
|UA     |8.147826086956522|
|UA     |8.265402843601896|
|AA     |8.610294117647058|
|AS     |7.413461538461538|
|AA     |8.5              |
|DL     |8.10135135135135 |
|DL     |8.779005524861878|
|DL     |8.08955223880597 |
|UA     |8.146666666666667|
|UA     |8.39090909090909 |
|AS     |8.274038461538462|
|DL     |8.53048780487805 |
|DL     |8.11111111111111 |
|UA     |8.336787564766839|
|DL     |8.434065934065934|
|UA     |8.088372093023256|
+-------+-----------------+
only showing top 20 rows



In [114]:
new_df.explain(
    extended=True,
)

== Parsed Logical Plan ==
'Filter 'AIRLINE IN (AA,AS,DL,UA)
+- Project [AIRLINE#1562, DIS_AIRTIME#1620]
   +- Project [YEAR#1558, MONTH#1559, DAY#1560, DAY_OF_WEEK#1561, AIRLINE#1562, FLIGHT_NUMBER#1563, TAIL_NUMBER#1564, ORIGIN_AIRPORT#1565, DESTINATION_AIRPORT#1566, SCHEDULED_DEPARTURE#1567, DEPARTURE_TIME#1568, DEPARTURE_DELAY#1569, TAXI_OUT#1570, WHEELS_OFF#1571, SCHEDULED_TIME#1572, ELAPSED_TIME#1573, AIR_TIME#1574, DISTANCE#1575, WHEELS_ON#1576, TAXI_IN#1577, SCHEDULED_ARRIVAL#1578, ARRIVAL_TIME#1579, ARRIVAL_DELAY#1580, DIVERTED#1581, ... 8 more fields]
      +- Filter MONTH#1559 IN (6,7,8)
         +- Relation [YEAR#1558,MONTH#1559,DAY#1560,DAY_OF_WEEK#1561,AIRLINE#1562,FLIGHT_NUMBER#1563,TAIL_NUMBER#1564,ORIGIN_AIRPORT#1565,DESTINATION_AIRPORT#1566,SCHEDULED_DEPARTURE#1567,DEPARTURE_TIME#1568,DEPARTURE_DELAY#1569,TAXI_OUT#1570,WHEELS_OFF#1571,SCHEDULED_TIME#1572,ELAPSED_TIME#1573,AIR_TIME#1574,DISTANCE#1575,WHEELS_ON#1576,TAXI_IN#1577,SCHEDULED_ARRIVAL#1578,ARRIVAL_TIME#1579,A